In [2]:
import torch

torch.cuda.device(0)

In [ ]:
%cd asset


In [ ]:
%cd /home/m25dehgh/simplification/Edit-Unsup-TS/src/asset
!/home/m25dehgh/anaconda3/envs/easenv/bin/easse evaluate --orig_sents_path asset.test.orig.2 --sys_sents_path asset.handgenerated --refs_sents_paths 'asset.testing.simp' -t custom -m 'sari,bleu' -q


In [ ]:
# effective 28
# non-effective 32

In [ ]:
import normalizing
print('one side of the armed conflicts is composed mainly of the sudanese military and the janjaweed , a sudanese militia group recruited mostly from the afro-arab abbala tribes of the northern rizeigat region in sudan .')
print(normalizing.all_norms('one side of the armed conflicts is composed mainly of the sudanese military and the janjaweed , a sudanese militia group recruited mostly from the afro-arab abbala tribes of the northern rizeigat region in sudan .'))

In [ ]:
from nltk.parse import CoreNLPParser

parser = CoreNLPParser(url='http://localhost:9000')
con_tree = list(parser.parse('one side of the armed conflicts is composed mainly of the sudanese military and the janjaweed, a sudanese militia group recruited mostly from the afro-arab abbala tribes of the northern rizeigat region in sudan.'.split()))
con_tree = list(parser.parse('the great dark spot is thought to represent a hole in the methane cloud deck of neptune.'.split()))
con_tree

In [ ]:
# parse = next(parser.raw_parse("I put the book in the box on the table."))
for i in con_tree:
    print(i)

In [ ]:
import os
from nltk.tree import Tree
from nltk.draw.tree import TreeView
t = Tree.fromstring('(S (NP this tree) (VP (V is) (AdjP pretty)))')
# t = con_tree[0]
TreeView(t)._cframe.print_to_file('output.ps')
os.system('convert output.ps output.png')

In [ ]:
from nltk import Tree
from nltk.draw.util import CanvasFrame
from nltk.draw import TreeWidget
from IPython.display import display
from IPython.display import Image
sentence = 'I saw a dog'
# parser = Parser()
parsed = parser.parse(sentence)
cf = CanvasFrame()
tc = TreeWidget(cf.canvas(),parsed)
cf.add_widget(tc,10,10) # (10,10) offsets
cf.print_to_file('tree.ps')
cf.destroy()
! convert tree.ps tree.png
! rm tree.ps

In [ ]:
type(con_tree[0])

In [2]:
# %cd /home/m25dehgh/simplification/Edit-Unsup-TS/

# wiki_orig_out = open('src/Wikilarge/output/RM+EX+LS+RO.txt', encoding='utf-8').read().split('\n')
root_path = '/home/m25dehgh/simplification/controllable-simplification/src/'
wiki_me_out = open(root_path + 'Wikilarge/output/simplifications.txt', encoding='utf-8').read().split('\n')

wiki_me_out 

new_wiki = []
for i in range(len(wiki_me_out)):
    if i % 8 == 2:
        new_wiki.append(wiki_me_out[i])

# new_wiki = new_wiki[-359:]
        
with open(root_path + '/Wikilarge/output/dl:false-par:1-asset-comp-simp-class.txt', "w") as file:
    for i in range(len(new_wiki)):
        file.write(new_wiki[i] + "\n")

In [4]:
new_wiki[-1]

'a string can vibrate in different ways and each form .'

# Be careful

In [ ]:
newsela_test_dst = []
for i in range(10):
    newsela_test_dst.append(open('/home/m25dehgh/simplification/datasets/asset/dataset/asset.test.simp.{}'.format(i), encoding='utf-8').read().split('\n'))

    newsela_test_dst[i] = [normalizing.all_norms(j) for j in newsela_test_dst[i]]


    with open('/home/m25dehgh/simplification/datasets/asset/dataset/asset.test.norm.simp.{}'.format(i), "w") as file:
        for j in range(len(newsela_test_dst[i])):
            file.write(newsela_test_dst[i][j] + "\n")
        


In [ ]:
newsela_test_dst = open('/home/m25dehgh/simplification/datasets/asset/dataset/asset.test.orig', encoding='utf-8').read().split('\n')

newsela_test_dst = [normalizing.all_norms(j) for j in newsela_test_dst]


with open('/home/m25dehgh/simplification/datasets/asset/dataset/asset.test.norm.orig', "w") as file:
    for j in range(len(newsela_test_dst)):
        file.write(newsela_test_dst[j] + "\n")

In [ ]:
valid_src = open('/home/m25dehgh/simplification/Edit-Unsup-TS/src/turkcorpus/test.8turkers.tok.norm', encoding='utf-8').read().split('\n')

In [ ]:
def score(f,s,model = 12):
    sum = 0
    for c in f:
        sum += int(c)
    for c in s:
        sum += int(c)
    return -sum

In [ ]:
# from queue import PriorityQueue
# pq = PriorityQueue() 
import heapq

pq = [] # Priority Queue
indx = ['0','1','2','3','4','5']
model = 12
beam_size = 3
used = {}
for i in range(len(indx)):
    for j in range(i+1, len(indx)):
        heapq.heappush(pq, (score(indx[i],indx[j],model), indx[i] + indx[j]))
        heapq.heappush(pq, (score(indx[j],indx[i],model), indx[j] + indx[i]))

        used.update({indx[i] + indx[j]: score(indx[i],indx[j],model)})
        used.update({indx[j] + indx[i]: score(indx[j],indx[i],model)})


bool_break = False
final_seq = ''
i = 0

while True:
    
#     best_seq = heapq.nsmallest(beam_size,pq, key=lambda s: s[0])

    best_seq = []
    for beam in range(beam_size):
        pair = heapq.heappop(pq)
        best_seq.append(pair)
        sequence = pair[1]
        if len(sequence) == 6:
            final_seq = pair
            bool_break = True
    


    if bool_break:
        break

        
    for pair in best_seq:
        sent_str = pair[1]
        indx = ['0','1','2','3','4','5']
        
        # Remove existing sentences in the current sequence in order to not be included in the future sequences
        for car in sent_str:
            indx.remove(car)
            
        for new_sent in indx:
            cand1 = sent_str + new_sent
            cand2 = new_sent + sent_str
            
            if cand1 not in used:
                score_cand1 = score(sent_str, new_sent, model)
                used.update({cand1: score_cand1})
                heapq.heappush(pq, (score_cand1, cand1))

            if cand2 not in used:
                score_cand2 = score(new_sent, sent_str, model)
                used.update({cand2: score_cand2})
                heapq.heappush(pq, (score_cand2, cand2))
             
        
final_seq

In [ ]:
# calculate the spearman's correlation between two variables
from numpy.random import rand
from numpy.random import seed
from scipy.stats import spearmanr
# seed random number generator
seed(1)
# prepare data
data1 = rand(10) * 20
data2 = data1 + (rand(10) * 10)
# calculate spearman's correlation
coef, p = spearmanr(data1, data2)
print('Spearmans correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

In [ ]:
from scipy.stats import spearmanr
data1 = [4,2,3,5,1]
data2 = [1,3,2,4,5]
spearmanr(data1, data2)

In [ ]:
heapq.heappop(b)

In [ ]:
# import heapq
kk = [1,12,3,8,15,20,2,100]
# heapq.heapify(kk)
# a.sort()
print(heapq.heappop(kk))
# kk

In [ ]:
a = '123'
for i in a:
    print(i == '1')